### Setup

apt install requirements

In [ ]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

: 

Clone repo and install dependencies

In [ ]:
%cd /content

## TODO: use when repo is public
# !git clone https://github.com/jannerm/diffuser.git
## backup for private repo
!curl -L -o diffuser.zip https://www.dropbox.com/s/fe1scd8m2jf7zvj/diffuser.tar?dl=1
!tar -xf diffuser.zip
## /TODO

%cd diffuser
%pip install -f https://download.pytorch.org/whl/torch_stable.html \
                typed-argument-parser \
                scikit-image==0.17.2 \
                scikit-video==1.1.11 \
                gitpython \
                einops \
                pillow \
                free-mujoco-py \
                gym \
                git+https://github.com/JannerM/d4rl.git@0e84a4d29db3ae501043215ce1d91843929f1949 \
                git+https://github.com/aravindr93/mjrl

%pip install -e .

: 

In [ ]:
## cythonize mujoco-py at first import
import diffuser.utils as utils

# import importlib
# importlib.reload(utils.colab)

: 

### Conditional sampling

In [ ]:
class Args:
  loadpath = 'logs/azure/hopper-medium-expert-v2/diffusion/H128_T100'
  diffusion_epoch = 'latest'
  n_samples = 4
  device = 'cuda:0'
    
args = Args()

: 

Load diffusion model from disk

In [ ]:
%cd /content/diffuser

diffusion_experiment = utils.load_diffusion(
    args.loadpath, epoch=args.diffusion_epoch)

dataset = diffusion_experiment.dataset
renderer = diffusion_experiment.renderer
model = diffusion_experiment.trainer.ema_model

: 

Initialize environment

In [ ]:
env = dataset.env
obs = env.reset()

: 

Run reverse diffusion

In [ ]:
observations = utils.colab.run_diffusion(
    model, dataset, obs, args.n_samples, args.device)
print(observations.shape)

: 

Render the samples

In [ ]:
sample = observations[-1]
utils.colab.show_sample(renderer, sample)

: 

Render the full reverse diffusion process for the first sample.

This renders `horizon * n_diffusion_steps` images, so takes a few minutes. Increasing `substep` will speed up the video generation, but cause it to skip frames.

In [ ]:
utils.colab.show_diffusion(renderer, observations[:,:1], substep=1)

: 

: 